In [1]:
import seaborn as sns
import numpy as np
import pandas as pd
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression

Instructions for updating:
Use the retry module or similar alternatives.
hdf5 is not supported on this machine (please install/reinstall h5py for optimal experience)


In [2]:
iris = sns.load_dataset('iris')
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [3]:
iris['species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [4]:
def neural_network_model(input_size, lr):
    network = input_data(shape=[None, input_size], name='state')
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 1024, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)
    
    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)
    
    
    network = fully_connected(network, 3, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=lr, loss='categorical_crossentropy', name='iris')
    
    model = tflearn.DNN(network, tensorboard_dir='log')
    
    return model
    

In [5]:
def labelize(record):
    if record == 'setosa':
        return [1, 0, 0]
    elif record == 'versicolor':
        return [0, 1, 0]
    elif record == 'virginica':
        return [0, 0, 1]      

In [6]:
x_train = iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']].values
y_train = list(map(labelize, iris['species']))

In [7]:
model = neural_network_model(4, 0.0001)
model.fit(x_train, y_train, n_epoch=1000, snapshot_step=5, show_metric=True, run_id='flower')

Training Step: 2999  | total loss: 0.36579 | time: 0.056s
| Adam | epoch: 1000 | loss: 0.36579 - acc: 0.9449 -- iter: 128/150
Training Step: 3000  | total loss: 0.33457 | time: 0.079s
| Adam | epoch: 1000 | loss: 0.33457 - acc: 0.9504 -- iter: 150/150
--


In [8]:
predictions = model.predict(x_train[50:])
predictions = list(map(lambda x: x.argmax(), predictions))

In [9]:
model.predict([[0.3, 0.1, 1, 0.1]])

array([[0.02412942, 0.96700025, 0.00887036]], dtype=float32)

In [10]:
model.save('iris_model')

INFO:tensorflow:/Users/kim/Projects/kim/python_projects/iris_model is not in all_model_checkpoint_paths. Manually adding it.
